In [0]:
!pip install fitz
!pip install PyMuPDF
!pip install transformers

     |████████████████████████████████| 71kB 3.1MB/s 
     |████████████████████████████████| 3.1MB 8.4MB/s 
     |████████████████████████████████| 5.8MB 40.3MB/s 
     |████████████████████████████████| 430kB 44.3MB/s 
     |████████████████████████████████| 235kB 45.7MB/s 
     |████████████████████████████████| 450kB 45.0MB/s 
     |████████████████████████████████| 92kB 11.4MB/s 
     |████████████████████████████████| 235kB 38.3MB/s 
     |████████████████████████████████| 51kB 7.0MB/s 
  Created wheel for pyxnat: filename=pyxnat-1.2.1.0.post3-cp36-none-any.whl size=71623 sha256=e0e00f5f368769aedb29e439da0e3a59b5f016eae7b08f20455420db897a60b3
  Stored in directory: /root/.cache/pip/wheels/98/46/71/7096c8f1537087e7628bdcc723f6e766880f8dde2667009371
  Created wheel for configobj: filename=configobj-5.0.6-cp36-none-any.whl size=34546 sha256=c709a59a0c7c2eff9b8bb3751391d0a974fcd1ea4387ec4aa29e25fcad00c4f2
  Stored in directory: /root/.cache/pip/wheels/f1/e4/16/4981ca97c2d65106b49861e

In [0]:
from transformers import BertModel, BertTokenizer
import torch
import numpy as np
import torch.nn as nn
import torch.nn.utils.rnn as rnn
from torch.utils.data import Dataset, DataLoader, TensorDataset
import fitz

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
DEVICE

'cuda'

In [0]:
ROOT = "/content/drive/My Drive/Capstone/"
doc = fitz.open(ROOT + "Cloud Computing Bible.pdf")

In [0]:
page = doc.loadPage(27)
dataset = page.getText()
dataset

In [0]:
import nltk.data

sent_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

print(len(sent_tokenizer.tokenize(dataset)))
for sentence in sent_tokenizer.tokenize(dataset):
  print(len(sentence))

24
154
88
89
76
74
163
218
26
264
17
113
67
84
106
188
166
112
116
127
78
104
175
134
190


In [0]:
class ModelDataset(Dataset):
    
    def __init__(self, data, sent_tokenizer, bert_tokenizer):
        sentences = sent_tokenizer.tokenize(data)

        sentence_tokens = []
        for sentence in sentences:
          tokens = torch.tensor(tokenizer.encode(sentence,max_length=25, pad_to_max_length=True, add_special_tokens=True))
          sentence_tokens.append(tokens.unsqueeze(0))
          #print(sentence)
          #print(tokenizer.tokenize(sentence))
          #print(len(tokenizer.tokenize(sentence)))
          #print(tokens)
          #print(len(tokens))
          #print("\n")
        
        self.data = torch.cat(sentence_tokens,dim=0)
        print(self.data.shape)

    def __getitem__(self,i):
        txt = self.data[i]
        
        return txt[:-1],txt[1:]
    
    def __len__(self):
        return self.data.size(0)


In [0]:
class ModelLanguageModel(nn.Module):

    def __init__(self,vocab_size,embed_size,hidden_size, nlayers):
        super(ModelLanguageModel,self).__init__()
        self.vocab_size=vocab_size
        self.embed_size = embed_size
        self.hidden_size = hidden_size
        self.nlayers=nlayers

        self.bert_layer = BertModel.from_pretrained('bert-base-uncased')
        self.embedding = nn.Embedding(vocab_size,embed_size)
        self.rnn = nn.LSTM(input_size = embed_size,hidden_size=hidden_size,num_layers=nlayers)
        
        self.scoring = nn.Linear(hidden_size,vocab_size)
        
    def forward(self,seq_batch, attn_masks): # N x L

        embed = self.bert_layer(seq_batch, attention_mask = attn_masks)
        print(embed.shape)

        embed = embed[0][:, 1:]

        #embed = self.embedding(seq_batch) #L x N x E
        hidden = None
        embed = embed.permute(1, 0, 2)
        output_lstm,hidden = self.rnn(embed,hidden) #L x N x H
        output_lstm_flatten = output_lstm.view(-1,self.hidden_size) #(L*N) x H
        output_flatten = self.scoring(output_lstm_flatten) #(L*N) x V

        batch_size = seq_batch.size(1)
        return output_flatten.view(-1,batch_size,self.vocab_size)


In [0]:
def train_epoch(model, optimizer, train_loader, val_loader):

    criterion = nn.CrossEntropyLoss()
    criterion = criterion.to(DEVICE)
    model.train()

    for batch_idx, (inputs,targets) in enumerate(train_loader):

        inputs = inputs.to(DEVICE)
        attn_masks = (inputs != 0).long()
        targets = targets.to(DEVICE).permute(1,0)
        outputs = model(inputs, attn_masks)
        
        loss = criterion(outputs.view(-1,outputs.size(2)),targets.view(-1))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        #if batch_idx % 50 == 0 and batch_idx != 0:
        print("batch_idx: " + str(batch_idx) + ", loss: " + str(loss.item()))

In [0]:
model = ModelLanguageModel(charcount,256,256,3)
model = model.to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(),lr=0.001, weight_decay=1e-6)

import nltk.data
sent_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
train_dataset = ModelDataset(dataset, sent_tokenizer, bert_tokenizer)
#val_dataset = ModelDataset(shakespeare_array[split:])
train_loader = DataLoader(train_dataset, shuffle=False, batch_size=2)
#val_loader = DataLoader(val_dataset, shuffle=False, batch_size=64, collate_fn = collate, drop_last=True)

In [0]:
for i in range(10):
    train_epoch(model, optimizer, train_loader)